### K MEANS CONSTRAINED

For install use:   pip install k-means-constrained

Documentation: https://joshlk.github.io/k-means-constrained/

In [1]:
import pandas as pd
import numpy as np
import geopandas
from k_means_constrained import KMeansConstrained
from scipy.spatial import ConvexHull, Delaunay

import requests
import random
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import folium

import warnings
warnings.filterwarnings('ignore')

In [2]:
# FUNCTION FOR CALCULATE POLYGON
def calculate_polygon(group, df):
    # filter data
    points = np.array(df.loc[df['region']==group][['lat','lon']])
    if points.shape[0]>2:
        # shape calculation
        hull = ConvexHull(points)
        hull_points = hull.simplices
        # return polygon shape
        polygon_shape = [[points[x,0], points[x,1]] for x in hull_points]
    else:
        # if can`t calculate return points
        polygon_shape = points
    return polygon_shape

In [3]:
# CREATE CODE GROUPS
codes_data = pd.read_excel('codes_base.xlsx').dropna()
codes_data['code group'] = codes_data['postal code'].apply(lambda x: str(x)[:4])
codes_data.head()

,postal code,lat,lon,code group
0,00-001,52.229852,21.006525,00-0
1,00-004,52.235602,21.008997,00-0
2,00-008,52.234537,21.009550,00-0
3,00-009,52.234699,21.010639,00-0
4,00-012,52.233598,21.011262,00-0


In [4]:
# CALCULATE CENTER OF GROUP AS MEAN OF MAX AND MIN GEOLOCATION
codes_data = codes_data.assign(lat_min = codes_data['lat'],lat_max = codes_data['lat'],
                              lon_min = codes_data['lon'], lon_max = codes_data['lon'])

group_codes_data = codes_data[['code group','lat_min','lat_max','lon_min','lon_max']]\
            .groupby(['code group'])\
            .agg({'lat_min':'min', 'lat_max':'max', 'lon_min':'min', 'lon_max':'max'})
group_codes_data = group_codes_data.assign(group_lat = (group_codes_data['lat_min']+group_codes_data['lat_max'])/2,
                                          group_lon = (group_codes_data['lon_min']+group_codes_data['lon_max'])/2)\
                                    .reset_index()\
                                    .drop(columns=['lat_min', 'lat_max', 'lon_min', 'lon_max'])
group_codes_data.head()

,code group,group_lat,group_lon
0,00-0,52.238210,21.011439
1,00-1,52.244515,20.995951
2,00-2,52.252114,21.005576
3,00-3,52.239691,21.024281
4,00-4,52.223751,21.034269


In [5]:
# GROUP CODES IN TO CLUSTERS WITH CONSTRAINS

# define k-means-constrained method
clf = KMeansConstrained(
     n_clusters=130,
     size_min=1,
     size_max=5,
     random_state=1)

group_codes_data['cluster'] = clf.fit_predict(group_codes_data[['group_lat','group_lon']])

In [6]:
# DEFINE REGION NAME
group_codes_data['region'] = group_codes_data['cluster'].apply(lambda x: 'REGION_'+str(x))

In [7]:
# CHECK GROUPS` SIZE
group_codes_data['region'].value_counts()

REGION_1      5
REGION_59     5
REGION_122    5
REGION_27     5
REGION_99     5
             ..
REGION_129    3
REGION_12     3
REGION_70     3
REGION_26     2
REGION_92     2
Name: region, Length: 130, dtype: int64

In [8]:
# ADD REGION TO POSTAL CODES
codes = pd.merge(codes_data, group_codes_data[['code group','region']], on=['code group'])\
        .drop(columns=['lat_min','lat_max','lon_min','lon_max'])

In [9]:
codes.head()

,postal code,lat,lon,code group,region
0,00-001,52.229852,21.006525,00-0,REGION_1
1,00-004,52.235602,21.008997,00-0,REGION_1
2,00-008,52.234537,21.009550,00-0,REGION_1
3,00-009,52.234699,21.010639,00-0,REGION_1
4,00-012,52.233598,21.011262,00-0,REGION_1


In [10]:
# CALCULATE SHAPE OF CODE GROUP BY GENERATE POLYGON FROM CENTERS OF POSTAL CODES USING CONVEX_HULL METHOD

In [11]:
# CALCULATE SHAPE FOR EACH CODES GROUP
codes['shape'] = codes['region'].apply(lambda x: calculate_polygon(x, codes))

In [12]:
codes.head()

,postal code,lat,lon,code group,region,shape
0,00-001,52.229852,21.006525,00-0,REGION_1,"[[[52.2131816, 52.19375094615384], [21.0041462..."
1,00-004,52.235602,21.008997,00-0,REGION_1,"[[[52.2131816, 52.19375094615384], [21.0041462..."
2,00-008,52.234537,21.009550,00-0,REGION_1,"[[[52.2131816, 52.19375094615384], [21.0041462..."
3,00-009,52.234699,21.010639,00-0,REGION_1,"[[[52.2131816, 52.19375094615384], [21.0041462..."
4,00-012,52.233598,21.011262,00-0,REGION_1,"[[[52.2131816, 52.19375094615384], [21.0041462..."


In [13]:
# PREPARE REGIONS SHAPES FOR MAP
cleaned_codes = codes[['region','shape']].drop_duplicates(subset='region').sort_values(by='region')

In [14]:
cleaned_codes.head()

,region,shape
5770,REGION_0,"[[[51.30898202719809, 51.39186320058309], [17...."
0,REGION_1,"[[[52.2131816, 52.19375094615384], [21.0041462..."
5523,REGION_10,"[[[51.12935884642857, 51.140787498795184], [16..."
1400,REGION_100,"[[[52.08371441764032, 52.0233929403734], [21.8..."
2271,REGION_101,"[[[53.71360854413408, 53.42220474739274], [22...."


In [15]:
# SHOW RESULTS ON MAP
colors = {0: "darkblue", 1: 'pink', 2: "orange", 3:"gray", 4: "red", 5: "green", 6: 'blue', 
        7:'darkgreen', 8:'white', 9:'purple', 10:'darkpurple', 11: 'beige', 12: 'yellow'}

created_map = folium.Map(location=[51.78,19.45], zoom_start=6)

# create codes layer on map
codes_layer = folium.FeatureGroup(name='CODES', col= 'black', show=True).add_to(created_map)
# prepade codes data from all data
for _, row in cleaned_codes.iterrows():
    try:
        shape_data = row['shape']
        polygon_shape = []
        for l in shape_data:
            polygon_shape.append([l[0][0], l[1][0]])
            polygon_shape.append([l[0][1], l[1][1]])
        
        color_for_code = random.randint(0,9)
        code_polygon = folium.Polygon(polygon_shape,
                                      color = colors[color_for_code], 
                                      weight=7, fill=True, 
                                      fill_color=colors[color_for_code],
                                      tooltip = row['region'])
        codes_layer.add_child(code_polygon)
    except:
        print(f'ERROR: {row["region"]}')
    
# add menu   
folium.map.LayerControl('topleft', collapsed= False).add_to(created_map)

created_map

In [16]:
# SAVE THE RESULTS
codes[['region','code group']].drop_duplicates().sort_values(by='region').to_excel('regiony_n_max_5.xlsx', index=False)
created_map.save('regiony_max_n_5.html')